# Play by Play Gatherer

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from urllib.request import urlopen
pd.set_option('display.max_colwidth', None)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
requestHead = {"User-Agent": "Chrome/47.0.2526.111"}

## General

In [3]:
def getSeasonIDs(num,online = True,prefix = None,post=True):
    if online:
        if num < 10:
            strnum = '0' + str(num)
        elif num >= 10:
            strnum = str(num)
        if num < 24:
            url = "http://sim-football.com/indexes/NSFLS"+strnum+"/GameResults.html"
        else:
            url = "http://sim-football.com/indexes/ISFLS"+strnum+"/GameResults.html"
        page = requests.get(url, headers = requestHead)
        soup = BeautifulSoup(page.text,'html.parser')
        
        if num < 5:
            gpwk = 3
        elif num == 5:
            gpwk = 4
        elif num <= 21:
            gpwk = 5
        elif num <= 24:
            gpwk = 6
        else:
            gpwk = 7
    else:
        with open(prefix+'/GameResults.html') as f:
            soup = BeautifulSoup(f,'html.parser')
    
    if num > 24:
        preseason = 28
        postseason = 7
    elif num > 21:
        preseason=24
        postseason=7
    elif num > 15:
        preseason = 20
        postseason= 7
    elif num > 1:
        preseason = 16
        postseason= 3
    else:
        preseason = 12
        postseason= 3
    
    if num <= 15:
        wks = 14
    elif num <= 22:
        wks = 13
    else:
        wks = 16
        
    wkList = np.repeat(range(1,wks),gpwk)
    
    pbplist = soup.find_all('a',href=re.compile('Logs'))
    pbpURLs = [p.get('href') for p in pbplist]
    if len(pbpURLs[preseason:-postseason]) == 55:
        preseason = preseason-1
    if post:
        idList = [p[5:].strip('.html') for p in pbpURLs[preseason:]]
    else:
        idList = [p[5:].strip('.html') for p in pbpURLs[preseason:-postseason]]
    return idList, wkList

In [6]:
sIDList = getSeasonIDs(S)
# gameID = sIDList[4]
# print(gameID)

In [5]:
gameID = 2334
S = 9

if S < 10:
    strnum = '0' + str(S)
elif S >= 10:
    strnum = str(S)

In [9]:
pagePBP = requests.get('http://sim-football.com/indexes/NSFLS%s/Logs/%s.html'%(strnum,gameID),headers=requestHead)
soupPBP = BeautifulSoup(pagePBP.content.decode('ISO-8859-1'),'lxml')
tablePBP = soupPBP.find_all('table',class_='Grid')[0]

In [11]:
tablePBP.find_all('tr')[15].find_all('td')[3].text.split(' - ')

['YKW', '42']

In [12]:
q = -2
pbpList = []

for row in tablePBP.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) != 5:
        q += 1
    elif len(cols) == 5:
        team = cols[0].find_all('img')[0]['src'][16:].strip('_s.png')
        textTime = cols[1].text
        minutes,seconds = cols[1].text.strip().split(':')
        secondsLeft = int(minutes)*60 + int(seconds)
        totTime = 15*60*(4-q) + secondsLeft
        downDist = cols[2].text
        if downDist == '':
            down = ''
            dist = ''
        elif downDist == '---':
            down = ''
            dist = ''
        else:
            down = int(downDist[0])
            dist = downDist[8:]
        loc = cols[3].text
        if loc == '':
            side = yard = ''
        else:
#             side = loc[:3]
#             yard = int(loc[-2:])
            side, yard = loc.split(' - ')
        play = cols[4].text
        pbpList.append((team,q,textTime,totTime,down,dist,side,yard,play))
    
pbpDF = pd.DataFrame(pbpList)
pbpDF.columns = ['teamID','Q','time','totTime','down','distance','side','yard','play']
pbpDF['gameID'] = gameID
pbpDF['S'] = S


In [13]:
pbpDF

,teamID,Q,time,totTime,down,distance,side,yard,play,gameID,S
0,5,1,15:00,3600,,,,,"Kickoff by Forfeit, N. deep into the endzone. Atwell, T. kneels for a touchback.",2334,9
1,5,1,15:00,3600,1,10,OCO,20,"Pass by Boss, M., complete to Westfield, B. for 27 yds. Tackle by Zhang, S..",2334,9
2,5,1,14:06,3546,1,10,OCO,47,"Pass by Boss, M., complete to Westfield, B. for 12 yds. Tackle by Tweed, B..",2334,9
3,5,1,13:36,3516,1,10,YKW,41,"Pass by Boss, M., complete to Crush, C. for 23 yds. Tackle by Lavelle, D..",2334,9
4,5,1,12:50,3470,1,10,YKW,18,"Rush by Yates, J. for a short gain. Tackle by (R) Brown, J..",2334,9
...,...,...,...,...,...,...,...,...,...,...,...
183,2,4,1:59,119,1,10,OCO,25,"Rush by Gore, B. for 1 yds. Tackle by Kane, M..",2334,9
184,2,4,1:25,85,2,9,OCO,24,"Rush by Gore, B. for 6 yds. Tackle by Winchester, A..",2334,9
185,2,4,0:51,51,3,3,OCO,18,"Rush by Gore, B. for 4 yds. Tackle by Wright Jr, G..",2334,9
186,2,4,0:22,22,1,10,OCO,14,"Rush by Rainey, R. for a short gain. Tackle by Brown, M..",2334,9


In [14]:
teamList = list(pbpDF['side'].unique())
print(teamList)
teamList = [t for t in teamList if t != '']
teamList

['', 'OCO', 'YKW']


['OCO', 'YKW']

In [15]:
'touchback' in pbpDF.iloc[0].play

True

In [16]:
if 'touchback' in pbpDF.iloc[0].play:
    pbpDF['homeTeam'] = teamList[1]
    pbpDF['awayTeam'] = teamList[0]
else:
    pbpDF['homeTeam'] = teamList[0]
    pbpDF['awayTeam'] = teamList[1]

In [17]:
pbpDF

,teamID,Q,time,totTime,down,distance,side,yard,play,gameID,S,homeTeam,awayTeam
0,5,1,15:00,3600,,,,,"Kickoff by Forfeit, N. deep into the endzone. Atwell, T. kneels for a touchback.",2334,9,YKW,OCO
1,5,1,15:00,3600,1,10,OCO,20,"Pass by Boss, M., complete to Westfield, B. for 27 yds. Tackle by Zhang, S..",2334,9,YKW,OCO
2,5,1,14:06,3546,1,10,OCO,47,"Pass by Boss, M., complete to Westfield, B. for 12 yds. Tackle by Tweed, B..",2334,9,YKW,OCO
3,5,1,13:36,3516,1,10,YKW,41,"Pass by Boss, M., complete to Crush, C. for 23 yds. Tackle by Lavelle, D..",2334,9,YKW,OCO
4,5,1,12:50,3470,1,10,YKW,18,"Rush by Yates, J. for a short gain. Tackle by (R) Brown, J..",2334,9,YKW,OCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2,4,1:59,119,1,10,OCO,25,"Rush by Gore, B. for 1 yds. Tackle by Kane, M..",2334,9,YKW,OCO
184,2,4,1:25,85,2,9,OCO,24,"Rush by Gore, B. for 6 yds. Tackle by Winchester, A..",2334,9,YKW,OCO
185,2,4,0:51,51,3,3,OCO,18,"Rush by Gore, B. for 4 yds. Tackle by Wright Jr, G..",2334,9,YKW,OCO
186,2,4,0:22,22,1,10,OCO,14,"Rush by Rainey, R. for a short gain. Tackle by Brown, M..",2334,9,YKW,OCO


In [19]:
pageBox = requests.get('http://sim-football.com/indexes/NSFLS%s/Boxscores/%s.html'%(strnum,gameID),headers=requestHead)
soupBox = BeautifulSoup(pageBox.content.decode('ISO-8859-1'),'lxml')
tableBox = soupBox.find_all('table',class_='Grid')[0]

In [20]:
scoreList = [(3600,0,0)]

q = 0
for row in tableBox.find_all('tr')[1:]:
    cols = row.find_all('td')
    if len(cols) != 6:
        q += 1
    else:
        time = cols[-4].text.split(':')
        secondsLeft = int(time[0])*60 + int(time[1])
        totTime = 15*60*(4-q) + secondsLeft
        
        awayScore = int(cols[-2].text)
        homeScore = int(cols[-1].text)
        
        scoreList.append((totTime,awayScore,homeScore))

scoreList.append((-901,awayScore,homeScore))

In [21]:
boxDF = pd.DataFrame(scoreList)
boxDF.columns = ['totTime','awayScore','homeScore']

In [22]:
pbpDF['awayTeam'] = tableBox.find_all('tr')[1:][0].find_all('td')[1].text
pbpDF['homeTeam'] = tableBox.find_all('tr')[1:][0].find_all('td')[2].text

In [23]:
def getTeams(S,teamID):
    if S < 5:
        teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'ARI','5':'OCO','6':'SJS','7':'PHI','8':'LV','9':'CHI','10':'AUS'}
    elif S == 5:
        teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'LV','9':'CHI','10':'AUS'}
    elif S <= 21:
        teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'NO','9':'CHI','10':'AUS'}
    else:
        teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'NO','9':'CHI','10':'AUS','11':'SAR','12':'HON'}
    return teamIDs[teamID]

def dist2goal(team,side,yard):
    if team == side:
        dist = 100 - int(yard)
    elif side != '':
        dist = int(yard)
    else:
        dist = ''
    return dist

def goal2go(distance,dist2goal):
    if distance == 'Goal':
        return int(dist2goal)
    elif distance == '':
        return ''
    elif distance == 'inches':
        return 1.
    else:
        return int(distance)

def getScore(boxDF,totTime):
    i = 0
    if int(totTime) > -901:
        while int(totTime) <= int(boxDF.loc[i+1]['totTime']):
            i += 1
        return boxDF.loc[i]['awayScore'],boxDF.loc[i]['homeScore']
    else:
        return boxDF.loc[len(boxDF)-1]['awayScore'],boxDF.loc[len(boxDF)-1]['homeScore']

def puntSide(play,side,awayTeam,homeTeam):
    if 'Punt' in play:
        if side == awayTeam:
            return homeTeam
        else:
            return awayTeam
    else:
        return side

def puntPoss(play,teamPoss,awayTeam,homeTeam):
    if 'Punt' in play:
        if teamPoss == awayTeam:
            return homeTeam
        else:
            return awayTeam
    else:
        return teamPoss

In [24]:
pbpDF['teamPoss'] = pbpDF.apply(lambda row: getTeams(S,row['teamID']),axis=1)

In [25]:
pbpDF['dist2goal'] = pbpDF.apply(lambda row: dist2goal(row['teamPoss'],row['side'],row['yard']),axis=1)

In [26]:
pbpDF['distance'] = pbpDF.apply(lambda row: goal2go(row['distance'],row['dist2goal']),axis=1)

In [27]:
pbpDF['side'] = pbpDF.apply(lambda row: puntSide(row['play'],row['side'],row['awayTeam'],row['homeTeam']),axis=1)
pbpDF['teamPoss'] = pbpDF.apply(lambda row: puntPoss(row['play'],row['teamPoss'],row['awayTeam'],row['homeTeam']),axis=1)

In [28]:
pbpDF['awayScore'] = pbpDF.apply(lambda row : getScore(boxDF,row['totTime'])[0],axis=1)
pbpDF['homeScore'] = pbpDF.apply(lambda row : getScore(boxDF,row['totTime'])[1],axis=1)

In [29]:
if pbpDF.iloc[-1]['totTime'] < 0:
    pbpDF['awayScore'].iloc[-1] = boxDF['awayScore'].iloc[-1]
    pbpDF['homeScore'].iloc[-1] = boxDF['homeScore'].iloc[-1]

In [30]:
pbpDF

,teamID,Q,time,totTime,down,distance,side,yard,play,gameID,S,homeTeam,awayTeam,teamPoss,dist2goal,awayScore,homeScore
0,5,1,15:00,3600,,,,,"Kickoff by Forfeit, N. deep into the endzone. Atwell, T. kneels for a touchback.",2334,9,YKW,OCO,OCO,,0,0
1,5,1,15:00,3600,1,10,OCO,20,"Pass by Boss, M., complete to Westfield, B. for 27 yds. Tackle by Zhang, S..",2334,9,YKW,OCO,OCO,80,0,0
2,5,1,14:06,3546,1,10,OCO,47,"Pass by Boss, M., complete to Westfield, B. for 12 yds. Tackle by Tweed, B..",2334,9,YKW,OCO,OCO,53,0,0
3,5,1,13:36,3516,1,10,YKW,41,"Pass by Boss, M., complete to Crush, C. for 23 yds. Tackle by Lavelle, D..",2334,9,YKW,OCO,OCO,41,0,0
4,5,1,12:50,3470,1,10,YKW,18,"Rush by Yates, J. for a short gain. Tackle by (R) Brown, J..",2334,9,YKW,OCO,OCO,18,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2,4,1:59,119,1,10,OCO,25,"Rush by Gore, B. for 1 yds. Tackle by Kane, M..",2334,9,YKW,OCO,YKW,25,13,28
184,2,4,1:25,85,2,9,OCO,24,"Rush by Gore, B. for 6 yds. Tackle by Winchester, A..",2334,9,YKW,OCO,YKW,24,13,28
185,2,4,0:51,51,3,3,OCO,18,"Rush by Gore, B. for 4 yds. Tackle by Wright Jr, G..",2334,9,YKW,OCO,YKW,18,13,28
186,2,4,0:22,22,1,10,OCO,14,"Rush by Rainey, R. for a short gain. Tackle by Brown, M..",2334,9,YKW,OCO,YKW,14,13,28


In [35]:
def getGameData2(S,gameID):
    if S < 10:
        strnum = '0' + str(S)
    elif S >= 10:
        strnum = str(S)
    
    pagePBP = requests.get('http://sim-football.com/indexes/NSFLS%s/Logs/%s.html'%(strnum,gameID),headers=requestHead)
    soupPBP = BeautifulSoup(pagePBP.content.decode('ISO-8859-1'),'lxml')
    tablePBP = soupPBP.find_all('table',class_='Grid')[0]
    
    q = -2
    pbpList = []

    for row in tablePBP.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) != 5:
            q += 1
        elif len(cols) == 5:
            team = cols[0].find_all('img')[0]['src'][16:].strip('_s.png')
            textTime = cols[1].text
            minutes,seconds = cols[1].text.strip().split(':')
            secondsLeft = int(minutes)*60 + int(seconds)
            totTime = 15*60*(4-q) + secondsLeft
            downDist = cols[2].text
            if downDist == '':
                down = ''
                dist = ''
            elif downDist == '---':
                down = ''
                dist = ''
            else:
                down = int(downDist[0])
                dist = downDist[8:]
            loc = cols[3].text
            if loc == '':
                side = yard = ''
            else:
    #             side = loc[:3]
    #             yard = int(loc[-2:])
                side, yard = loc.split(' - ')
            play = cols[4].text
            pbpList.append((team,q,textTime,totTime,down,dist,side,yard,play))

    pbpDF = pd.DataFrame(pbpList)
    pbpDF.columns = ['teamID','Q','time','totTime','down','distance','side','yard','play']
    pbpDF['gameID'] = gameID
    pbpDF['S'] = S
        
    teamList = list(pbpDF['side'].unique())
    teamList = [t for t in teamList if t != '']
    
    pageBox = requests.get('http://sim-football.com/indexes/NSFLS%s/Boxscores/%s.html'%(strnum,gameID),headers=requestHead)
    soupBox = BeautifulSoup(pageBox.content.decode('ISO-8859-1'),'lxml')
    tableBox = soupBox.find_all('table',class_='Grid')[0]
    pbpDF['awayTeam'] = tableBox.find_all('tr')[1:][0].find_all('td')[1].text
    pbpDF['homeTeam'] = tableBox.find_all('tr')[1:][0].find_all('td')[2].text
    
    scoreList = [(3600,0,0)]

    q = 0
    for row in tableBox.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) != 6:
            q += 1
        else:
            time = cols[-4].text.split(':')
            secondsLeft = int(time[0])*60 + int(time[1])
            totTime = 15*60*(4-q) + secondsLeft

            awayScore = int(cols[-2].text)
            homeScore = int(cols[-1].text)

            scoreList.append((totTime,awayScore,homeScore))

    scoreList.append((-901,awayScore,homeScore))
    
    boxDF = pd.DataFrame(scoreList)
    boxDF.columns = ['totTime','awayScore','homeScore']
    
    pbpDF['teamPoss'] = pbpDF.apply(lambda row: getTeams(S,row['teamID']),axis=1)
    pbpDF['dist2goal'] = pbpDF.apply(lambda row: dist2goal(row['teamPoss'],row['side'],row['yard']),axis=1)
    pbpDF['distance'] = pbpDF.apply(lambda row: goal2go(row['distance'],row['dist2goal']),axis=1)
    pbpDF['side'] = pbpDF.apply(lambda row: puntSide(row['play'],row['side'],row['awayTeam'],row['homeTeam']),axis=1)
    pbpDF['teamPoss'] = pbpDF.apply(lambda row: puntPoss(row['play'],row['teamPoss'],row['awayTeam'],row['homeTeam']),axis=1)
    pbpDF['awayScore'] = pbpDF.apply(lambda row : getScore(boxDF,row['totTime'])[0],axis=1)
    pbpDF['homeScore'] = pbpDF.apply(lambda row : getScore(boxDF,row['totTime'])[1],axis=1)
    if pbpDF.iloc[-1]['totTime'] < 0:
        pbpDF['awayScore'].iloc[-1] = boxDF['awayScore'].iloc[-1]
        pbpDF['homeScore'].iloc[-1] = boxDF['homeScore'].iloc[-1]
    
    return pbpDF

In [36]:
getGameData2(9,2334)

,teamID,Q,time,totTime,down,distance,side,yard,play,gameID,S,awayTeam,homeTeam,teamPoss,dist2goal,awayScore,homeScore
0,5,1,15:00,3600,,,,,"Kickoff by Forfeit, N. deep into the endzone. Atwell, T. kneels for a touchback.",2334,9,OCO,YKW,OCO,,0,0
1,5,1,15:00,3600,1,10,OCO,20,"Pass by Boss, M., complete to Westfield, B. for 27 yds. Tackle by Zhang, S..",2334,9,OCO,YKW,OCO,80,0,0
2,5,1,14:06,3546,1,10,OCO,47,"Pass by Boss, M., complete to Westfield, B. for 12 yds. Tackle by Tweed, B..",2334,9,OCO,YKW,OCO,53,0,0
3,5,1,13:36,3516,1,10,YKW,41,"Pass by Boss, M., complete to Crush, C. for 23 yds. Tackle by Lavelle, D..",2334,9,OCO,YKW,OCO,41,0,0
4,5,1,12:50,3470,1,10,YKW,18,"Rush by Yates, J. for a short gain. Tackle by (R) Brown, J..",2334,9,OCO,YKW,OCO,18,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2,4,1:59,119,1,10,OCO,25,"Rush by Gore, B. for 1 yds. Tackle by Kane, M..",2334,9,OCO,YKW,YKW,25,13,28
184,2,4,1:25,85,2,9,OCO,24,"Rush by Gore, B. for 6 yds. Tackle by Winchester, A..",2334,9,OCO,YKW,YKW,24,13,28
185,2,4,0:51,51,3,3,OCO,18,"Rush by Gore, B. for 4 yds. Tackle by Wright Jr, G..",2334,9,OCO,YKW,YKW,18,13,28
186,2,4,0:22,22,1,10,OCO,14,"Rush by Rainey, R. for a short gain. Tackle by Brown, M..",2334,9,OCO,YKW,YKW,14,13,28
